In [61]:
import os
import codecs
import glob
import numpy as np
import re

In [2]:
# read file
def read_files():
    all_information = np.zeros((5, 944, 1683))
    os.getcwd()
    os.chdir("/Users/audrey/Documents/GitHub/Differential-Privacy/data/train")
    '''
    movie is the dictionary of the all the movie information
    the format is :
    {movie_id : [ [customer_id, ranking, date]]} 
    '''
    pattern = re.compile(r'\d+') #find numbers
    files = glob.glob('*.txt')

    index = 0
    for filename in files:
        f = codecs.open(filename, 'r', encoding='utf-8')
        for line in f:
                information = pattern.findall(line)
                all_information[index][int(information[0])][int(information[1])]=int(information[2])
        index += 1

    return all_information

In [3]:
# Movie Effects
def getMAvg(df):
    GSum = np.sum(df)
    GCnt = np.count_nonzero(df)
    G = GSum/GCnt
    
    beta = 0.1
    MSum = np.sum(df,axis=0) #the number of ratings for each movie
    MCnt = np.count_nonzero(df,axis=0)    
    # produce a stabilized per-movie average rating 
    MAvg = (MSum + beta*G)/(MCnt + beta)
    return MAvg
 
# User Effects
def getUAvg(df,MAvg):
    GSum = np.sum(df)
    GCnt = np.count_nonzero(df)
    G = GSum/GCnt
    
    beta = 0.1
    USum = np.sum(df-MAvg,axis=1) + beta*G
    c_u = np.count_nonzero(df,axis=1) #the number of movies for each user
    UAvg = USum/(c_u + beta)
    return UAvg

def clamp(df,UAvg,bound):
    #lower the sensitivity of the measurements
    df_hat = (df.transpose()-UAvg).transpose()
    df_hat[ df_hat < -bound ] = -bound
    df_hat[ df_hat > bound ] = bound
    return df_hat

In [51]:
# Covariance Matrix
def getCovWgt(cdf,df):
    w_u =  1/np.count_nonzero(df,axis=1) # 1/ ||e_u||
    w_u = np.repeat(w_u, np.shape(df)[1], axis=0)
    w_u = np.reshape(w_u,(np.shape(df)[0], np.shape(df)[1])).transpose()
    
    Cov = np.sum(w_u * cdf.transpose(),axis=1)  #ratings for each movie with a shape of (m,1)
    
    e_u = df*(df>0)
    e_u[e_u>0] = 1
    
    Wgt = np.sum(w_u * e_u.transpose(), axis=1) 
    

In [4]:
# user item rating
dfs = read_files()


In [5]:
df = dfs[0]
MAvg = getMAvg(df)
UAvg = getUAvg(df,MAvg)
cdf = clamp(df,UAvg,0.09)

In [50]:
arr=np.array([1,2,3,4,5,7,8])
arr[arr < 3] = 3
arr[arr > 5] = 7
print(arr)

[3 3 3 4 5 7 7]


In [56]:
x = np.array([1,2,3])
print(x)
y = np.array([[1,2,3,4,5],[2,3,4,5,6],[3,4,5,6,7]])
print(y)
z = x*y
print(z)

[1 2 3]
[[1 2 3 4 5]
 [2 3 4 5 6]
 [3 4 5 6 7]]


ValueError: operands could not be broadcast together with shapes (3,) (3,5) 

In [53]:
a = np.array([1,2,3])
b = np.repeat(a,4,axis=0)
print(b)
print(np.reshape(b,(3,4)).transpose())

[1 1 1 1 2 2 2 2 3 3 3 3]
[[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]]
